# Discount margin calculation

(Based on [two](http://quant.stackexchange.com/questions/8965/) [questions](https://quant.stackexchange.com/questions/37705/) by _Stack Exchange_ users HookahBoy and Kyle. Thanks!)

In [1]:
from QuantLib import *

In [2]:
today = Date(8, October, 2014)
Settings.instance().evaluationDate = today

#### The question

Given a floating-rate bond price, we want to find the corresponding discount margin.  This is one in a class of similar problems: we have a calculation which is not immediate to do directly, but is straightforward to do in the opposite direction; in this case, find the price of a bond when discounting its coupons at a spread over LIBOR.

The general idea is to implement the inverse calculation (DM to price) and then to use a solver to determine the correct input given the result. First, we build the bond.

In [3]:
forecast_curve = RelinkableYieldTermStructureHandle()
discount_curve = RelinkableYieldTermStructureHandle()

In [4]:
index = Euribor6M(forecast_curve)

In [5]:
issueDate = Date(13,October,2014)
maturityDate = Date(13,October,2024)

schedule = Schedule(issueDate, maturityDate,
                    Period(Semiannual), TARGET(), Following, Following,
                    DateGeneration.Backward, False)

In [6]:
bond = FloatingRateBond(settlementDays = 3,
                        faceAmount = 100,
                        schedule = schedule,
                        index = index,
                        paymentDayCounter = Actual360(),
                        paymentConvention = Following,
                        fixingDays = index.fixingDays(),
                        gearings = [],
                        spreads = [],
                        caps= [],
                        floors = [],
                        inArrears = False,
                        redemption = 100.0,
                        issueDate = issueDate)
bond.setPricingEngine(DiscountingBondEngine(discount_curve))

Now we link the forecast curve to the current Euribor curve (whatever that is; I'm using a flat one as an example, but it could as well be a real one)...

In [7]:
forecast_curve.linkTo(FlatForward(0, TARGET(), 0.002, Actual360()))

...and the discount curve to the Euribor curve plus the discount margin.

In [8]:
DM = SimpleQuote(0.0)
discount_curve.linkTo(ZeroSpreadedTermStructure(forecast_curve,
                                                QuoteHandle(DM)))

Setting a value to the DM quote will affect the bond price: this gives us the knob to manipulate in order to find the solution of our problem.

In [9]:
print(bond.cleanPrice())

100.00000000000001


In [10]:
DM.setValue(0.001)
print(bond.cleanPrice())

98.99979030764418


To invert the calculation, we encapsulate the above into a function.  The Python language makes it easier to write it in a general way; the function below takes the target price, and returns another function that takes a value for the discount margin and returns the difference between the corresponding price and the target.  In C++, we would create a function object taking the target price in its constructor and returning the difference from its `operator()`.

In [11]:
def F(price):
    def _f(s):
        DM.setValue(s)
        return bond.cleanPrice() - price
    return _f

In [12]:
f = F(98.9997903076)
print(f(0.0))
print(f(0.002))

1.00020969240002
-0.9901429992548856


We want to find the value of the discount margin that causes the calculated price to equal the target price, that is, that causes the error to be 0; and for that, we can use a solver.

In [13]:
margin = Brent().solve(F(99.6), 1e-8, 0.0, 1e-4)
print(margin)

0.00039870328652332745


We can verify that this works by setting the margin to the returned value and checking that the bond price equals the input:

In [14]:
DM.setValue(margin)
print(bond.cleanPrice())

99.59999988275108


However, note that the spread above is continuously compounded. You might want to see the discount margin in the same units as the index fixings:

In [15]:
value_date = index.valueDate(today)
maturity_date = index.maturityDate(value_date)
print(InterestRate(margin, discount_curve.dayCounter(),
                   Continuous, NoFrequency)
      .equivalentRate(index.dayCounter(),
                      Simple, index.tenor().frequency(),
                      value_date, maturity_date))

0.039874 % Actual/360 simple compounding


#### Not just for bonds

The approach I described can be generalized to any problem in this class. Here I'll use it to get the implied volatility of an Asian option: first I'll create the instrument...

In [16]:
exerciseDate = today + Period(1,Years)
fixingDates = [ today + Period(n,Months) for n in range(1,12) ]
option = DiscreteAveragingAsianOption(Average.Arithmetic,
                                      0.0, 0,
                                      fixingDates,
                                      PlainVanillaPayoff(Option.Call, 100.0),
                                      EuropeanExercise(exerciseDate))

...and an engine, taking care of writing the input volatility as a quote.

In [17]:
sigma = SimpleQuote(0.20)

riskFreeCurve = FlatForward(0, TARGET(), 0.01, Actual360())
volatility = BlackConstantVol(0, TARGET(), QuoteHandle(sigma), Actual360())

process = BlackScholesProcess(QuoteHandle(SimpleQuote(100.0)),
                              YieldTermStructureHandle(riskFreeCurve),
                              BlackVolTermStructureHandle(volatility))

In [18]:
option.setPricingEngine(MCDiscreteArithmeticAPEngine(process, "pseudorandom",
                                                     requiredSamples=1000,
                                                     seed=42))

Now we can use the same technique as above: the function below takes a target price and returns a function from the volatility to the pricing error:

In [19]:
def F(price):
    def _f(v):
        sigma.setValue(v)
        return option.NPV() - price
    return _f

Using a solver, we can invert it to solve for any price:

In [20]:
print(Brent().solve(F(5.0), 1e-8, 0.20, 1e-4))

0.20081193864526342


In [21]:
print(Brent().solve(F(6.0), 1e-8, 0.20, 1e-4))

0.24362397543255393
